In [1]:
import argparse
import math
import h5py
import numpy as np
import tensorflow as tf
import socket

import os
import sys

In [2]:
BASE_DIR = "/data/code3/mil_pointnet/sem_seg"#os.path.dirname(os.path.abspath(__file__))
ROOT_DIR = os.path.dirname(BASE_DIR)

In [3]:
BASE_DIR, ROOT_DIR

('/data/code3/mil_pointnet/sem_seg', '/data/code3/mil_pointnet')

In [4]:
sys.path.append(BASE_DIR)
sys.path.append(ROOT_DIR)

In [5]:
os.path.join(ROOT_DIR, 'utils')

'/data/code3/mil_pointnet/utils'

In [6]:
sys.path.append(os.path.join(ROOT_DIR, 'utils'))

In [7]:
import provider

In [8]:
from model import *

Namespace(batch_size=24, decay_rate=0.5, decay_step=300000, gpu=0, learning_rate=0.001, log_dir='log', max_epoch=50, momentum=0.9, num_point=4096, optimizer='adam', test_area=6)

In [9]:
# parmas:
BATCH_SIZE = 24 # help='Batch Size during training [default: 24]')
NUM_POINT = 4096 #, help='Point number [default: 4096]')
MAX_EPOCH = 50 #help='Epoch to run [default: 50]')
BASE_LEARNING_RATE = 0.001 #, help='Initial learning rate [default: 0.001]'
GPU_INDEX = 0 #, help='GPU to use [default: GPU 0]')# 
MOMENTUM = 0.9 #, help='Initial learning rate [default: 0.9]')
OPTIMIZER = 'adam' #, help='adam or momentum [default: adam]')
DECAY_STEP = 300000 #, help='Decay step for lr decay [default: 300000]')
DECAY_RATE = 0.5 #, help='Decay rate for lr decay [default: 0.5]')FLAGS.decay_rate
TEST_AREA = 6

In [10]:
LOG_DIR = 'log180617'#, help='Log dir [default: log]')
if not os.path.exists(LOG_DIR): os.mkdir(LOG_DIR)
os.system('cp model.py %s' % (LOG_DIR)) # bkp of model def
os.system('cp train.py %s' % (LOG_DIR)) # bkp of train procedure
LOG_FOUT = open(os.path.join(LOG_DIR, 'log_train.txt'), 'w')
#LOG_FOUT.write(str(FLAGS)+'\n')

In [11]:
MAX_NUM_POINT = 4096
NUM_CLASSES = 13

# batch_norm params
BN_INIT_DECAY = 0.5
BN_DECAY_DECAY_RATE = 0.5
#BN_DECAY_DECAY_STEP = float(DECAY_STEP * 2)
BN_DECAY_DECAY_STEP = float(DECAY_STEP)
BN_DECAY_CLIP = 0.99


In [12]:
socket.gethostname()

'milton-ThinkCentre-M93p'

In [13]:
HOSTNAME = socket.gethostname()

In [14]:
ALL_FILES = provider.getDataFiles('indoor3d_sem_seg_hdf5_data/all_files.txt')
room_filelist = [line.rstrip() for line in open('indoor3d_sem_seg_hdf5_data/room_filelist.txt')]

In [35]:
len(room_filelist)

23585

In [15]:
ALL_FILES

['indoor3d_sem_seg_hdf5_data/ply_data_all_0.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_1.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_2.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_3.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_4.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_5.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_6.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_7.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_8.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_9.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_10.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_11.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_12.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_13.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_14.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_15.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_16.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_17.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_18.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_19.h5',
 'indoor3d

In [16]:
"""def load_h5(h5_filename):
    f = h5py.File(h5_filename)
    data = f['data'][:]
    label = f['label'][:]
    return (data, label)"""
f = h5py.File(ALL_FILES[0])
data = f['data'][:]
label = f['label'][:]

In [24]:
data.shape

(585, 4096, 9)

In [25]:
label.shape

(585, 4096)

In [31]:
np.max(label[:,:]), np.min(label[:,:])

(12, 0)

In [23]:
for i in range(len(ALL_FILES)):
    f = h5py.File(ALL_FILES[i])
    data = f['data'][:]
    label = f['label'][:]
    print(data.shape, label.shape)

((1000, 4096, 9), (1000, 4096))
((1000, 4096, 9), (1000, 4096))
((1000, 4096, 9), (1000, 4096))
((1000, 4096, 9), (1000, 4096))
((1000, 4096, 9), (1000, 4096))
((1000, 4096, 9), (1000, 4096))
((1000, 4096, 9), (1000, 4096))
((1000, 4096, 9), (1000, 4096))
((1000, 4096, 9), (1000, 4096))
((1000, 4096, 9), (1000, 4096))
((1000, 4096, 9), (1000, 4096))
((1000, 4096, 9), (1000, 4096))
((1000, 4096, 9), (1000, 4096))
((1000, 4096, 9), (1000, 4096))
((1000, 4096, 9), (1000, 4096))
((1000, 4096, 9), (1000, 4096))
((1000, 4096, 9), (1000, 4096))
((1000, 4096, 9), (1000, 4096))
((1000, 4096, 9), (1000, 4096))
((1000, 4096, 9), (1000, 4096))
((1000, 4096, 9), (1000, 4096))
((1000, 4096, 9), (1000, 4096))
((1000, 4096, 9), (1000, 4096))
((585, 4096, 9), (585, 4096))


In [32]:
# Load ALL data
data_batch_list = []
label_batch_list = []
# Add all the h5 files into data_batch_list, and label_batch_list
for h5_filename in ALL_FILES:
    data_batch, label_batch = provider.loadDataFile(h5_filename)
    data_batch_list.append(data_batch)
    label_batch_list.append(label_batch) 
data_batches = np.concatenate(data_batch_list, 0)
label_batches = np.concatenate(label_batch_list, 0)
print(data_batches.shape)
print(label_batches.shape)

In [34]:
data_batches.shape, label_batches.shape

((23585, 4096, 9), (23585, 4096))

In [36]:
test_area = 'Area_'+str(TEST_AREA)
train_idxs = []
test_idxs = []
for i,room_name in enumerate(room_filelist): # 23,585 files
    # use one area for testing, and the other five areas for training.
    if test_area in room_name:
        test_idxs.append(i)
    else:
        train_idxs.append(i)

In [38]:
len(train_idxs), len(test_idxs)

(20291, 3294)

In [39]:
train_data = data_batches[train_idxs,...]
train_label = label_batches[train_idxs]
test_data = data_batches[test_idxs,...]
test_label = label_batches[test_idxs]
print(train_data.shape, train_label.shape)
print(test_data.shape, test_label.shape)

((20291, 4096, 9), (20291, 4096))
((3294, 4096, 9), (3294, 4096))


In [ ]:
# construct placeholder
def placeholder_inputs(batch_size, num_point):
    pointclouds_pl = tf.placeholder(tf.float32,
                                     shape=(batch_size, num_point, 9))
    labels_pl = tf.placeholder(tf.int32,
                                shape=(batch_size, num_point))
    return pointclouds_pl, labels_pl

In [41]:
pointclouds_pl, labels_pl = placeholder_inputs(BATCH_SIZE, NUM_POINT) # 24, 4096

In [42]:
is_training_pl = tf.placeholder(tf.bool, shape=()) # to determine which stage is

In [44]:
def get_bn_decay(batch):
    bn_momentum = tf.train.exponential_decay(
                      BN_INIT_DECAY,
                      batch*BATCH_SIZE,
                      BN_DECAY_DECAY_STEP,
                      BN_DECAY_DECAY_RATE,
                      staircase=True)
    bn_decay = tf.minimum(BN_DECAY_CLIP, 1 - bn_momentum)
    return bn_decay

In [45]:
# Note the global_step=batch parameter to minimize. 
# That tells the optimizer to helpfully increment the 'batch' parameter for you every time it trains.
batch = tf.Variable(0)
bn_decay = get_bn_decay(batch)
tf.summary.scalar('bn_decay', bn_decay) # let tensorboard to visualize bn_decay

<tf.Tensor 'bn_decay:0' shape=() dtype=string>

In [47]:
BATCH_SIZE, NUM_POINT

(24, 4096)

In [61]:
pointclouds_pl.shape

TensorShape([Dimension(24), Dimension(4096), Dimension(9)])

In [55]:
pointclouds_pl.get_shape()[0].value, pointclouds_pl.get_shape()[1].value, pointclouds_pl.get_shape()[2].value

(24, 4096, 9)

In [58]:
input_image = tf.expand_dims(pointclouds_pl, -1)

In [60]:
input_image.get_shape()

TensorShape([Dimension(24), Dimension(4096), Dimension(9), Dimension(1)])

In [62]:
# Construct model
# input: pointclouds_pl: 24, 4096, 9, : 24 box, sample 4096 points in each box, 
#        use 9-dim feature for each point in segmentation task.
batch_size = pointclouds_pl.get_shape()[0].value # 24
num_point = pointclouds_pl.get_shape()[1].value # 4096

input_image = tf.expand_dims(point_cloud, -1) # expand as (24, 4096, 9, 1)

In [ ]:
# CONV
# kernel_size=[1,9], "9" to convolve all 9-dim feature
net = tf_util.conv2d(inputs=input_image, num_output_channels=64, kernel_size=[1,9], 
                     padding='VALID', stride=[1,1], bn=True, is_training=is_training, 
                     scope='conv1', bn_decay=bn_decay)
net = tf_util.conv2d(inputs=net, num_output_channels=64, kernel_size=[1,1], padding='VALID', 
                     stride=[1,1], bn=True, is_training=is_training, scope='conv2', 
                     bn_decay=bn_decay)
net = tf_util.conv2d(inputs=net, num_output_channels=64, kernel_size=[1,1], padding='VALID', 
                     stride=[1,1], bn=True, is_training=is_training, scope='conv3', 
                     bn_decay=bn_decay)
net = tf_util.conv2d(inputs=net, num_output_channels=128, kernel_size=[1,1], padding='VALID', 
                     stride=[1,1], bn=True, is_training=is_training, scope='conv4', 
                     bn_decay=bn_decay)

In [65]:
type(net)

NameError: name 'net' is not defined

In [63]:
batch_size, num_point

(24, 4096)

In [ ]:
# Get model and loss 
pred = get_model(pointclouds_pl, is_training_pl, bn_decay=bn_decay)
loss = get_loss(pred, labels_pl)
tf.summary.scalar('loss', loss)